In [1]:
%matplotlib inline
import numpy as np

In [2]:
def top_k(C,c_id,cid2cname,topk=5):
    norms = np.apply_along_axis(np.linalg.norm, 0, C)
    C_norm = C / norms.reshape(-1,1)
    c_vec=C_norm[c_id]
    sim = np.dot(C_norm,c_vec)
    nearest = (-sim).argsort()[1:topk+1]
    results=[(cid2cname[nearest[k]],sim[nearest[k]]) for k in xrange(topk)]
    return results

def top_k_vec(C,vec,cid2cname,topk=5):
    C_norm=normalize(C)
    vec_norm=vec/np.linalg.norm(vec)
    sim = np.dot(C_norm,vec_norm)
    nearest = (-sim).argsort()[0:topk]
    results=[(cid2cname[nearest[k]],sim[nearest[k]]) for k in xrange(topk)]
    return results